In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install imbalanced-learn

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
import math

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

def to_categorical(y, nb_classes=None):
    '''Convert class vector (integers from 0 to nb_classes)
    to binary class matrix, for use with categorical_crossentropy.
    '''
    y = np.array(y, dtype='int')
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    for i in range(len(y)):
        Y[i, y[i]] = 1.
    return Y


def calculate_performace(test_num, pred_y, labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] == 1:
            if labels[index] == pred_y[index]:
                tp = tp + 1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn + 1
            else:
                fp = fp + 1

    acc = float(tp + tn) / test_num
    precision = float(tp) / (tp + fp + 1e-06)
    npv = float(tn) / (tn + fn + 1e-06)
    sensitivity = float(tp) / (tp + fn + 1e-06)
    specificity = float(tn) / (tn + fp + 1e-06)
    mcc = float(tp * tn - fp * fn) / (math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) + 1e-06)
    f1 = float(tp * 2) / (tp * 2 + fp + fn + 1e-06)
    return acc, precision, npv, sensitivity, specificity, mcc, f1


In [3]:
import scipy.io as sio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import scale,StandardScaler
from keras.layers import Dense, Input,Dropout
from tensorflow.keras.models import Sequential, model_from_json
from keras.models import Model
import os
####################################################################

#Load data
# path = "/content/drive/MyDrive/Watashara_Projects/ACE (Peptide)/"
# data_ = pd.read_csv(path + 'EN_All_clean.csv',header = None)

# data_np = np.array(data_)
# data = data_np[:, :]
# # minmax = MinMaxScaler()
# # data = minmax.fit(data_np)

# label1=np.ones((394,1))#Value can be changed
# label2=np.zeros((626,1))
# labels=np.append(label1,label2)

# from imblearn.over_sampling import SMOTE, BorderlineSMOTE
# oversample = SMOTE()
# X, y = oversample.fit_resample(data, labels)
# # y = labels


# X_train, X_ind, y_train, y_ind = train_test_split(X, y, test_size=0.2, random_state=42)

# print(X_train.shape)
# print(X_ind.shape)
# print(y_train.shape)
# print(y_ind.shape)



path = "/content/drive/MyDrive/Watashara_Projects/ACE/"
train_data = pd.read_csv(path + 'selected_features_allfeat/XtrainData_wSMOTE.csv')


X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values

print(np.shape(X_train))

X = np.array(X_train,dtype=float)
[m1,n1] = np.shape(X)
# y = np.array(y_train)

# X =
y = y_train

fldr_path = path+'Update_results/DNN_ACE/'
if not os.path.exists(fldr_path):
    os.makedirs(fldr_path)

[m,n]=np.shape(X)
sepscores = []
sepscores_ = []
ytest=np.ones((1,2))*0.5
yscore=np.ones((1,2))*0.5
def get_con_model():
    input_1 = Input(shape=(n,), name='Protein')
    protein_input1 = Dense(int(n*2), activation='relu', name='High_dim_feature_1')(input_1)
    protein_input1=Dropout(0.5)(protein_input1)
    protein_input1 = Dense(int(n), activation='relu', name='High_dim_feature_2')(protein_input1)
    protein_input1=Dropout(0.5)(protein_input1)
    # protein_input1 = Dense(int(n/2), activation='relu',  name='High_dim_feature_3')(protein_input1)
    # protein_input1=Dropout(0.5)(protein_input1)
    output = Dense(int(n/4), activation='relu',  name='High_dim_feature')(protein_input1)
    outputs = Dense(2, activation='softmax', name='output')(output)
    model = Model(inputs= input_1, outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model
model=get_con_model()

skf= StratifiedKFold(n_splits=5)
for i, (train, test) in enumerate (skf.split(X,y)):
    y_train=to_categorical(y[train])
    cv_clf =get_con_model()
    hist=cv_clf.fit(X[train],
                    y_train,
                    epochs=15)
    y_test=to_categorical(y[test])
    ytest=np.vstack((ytest,y_test))
    y_test_tmp=y[test]
    y_score=cv_clf.predict(X[test])
    yscore=np.vstack((yscore,y_score))
    fpr, tpr, _ = roc_curve(y_test[:,0], y_score[:,0])
    roc_auc = auc(fpr, tpr)
    y_class= categorical_probas_to_classes(y_score)
    acc, precision,npv, sensitivity, specificity, mcc,f1 = calculate_performace(len(y_class), y_class, y_test_tmp)
    sepscores.append([acc, precision,npv, sensitivity, specificity, mcc,f1,roc_auc])
    print('DNN:acc=%f,precision=%f,npv=%f,sensitivity=%f,specificity=%f,mcc=%f,f1=%f,roc_auc=%f'
          % (acc, precision,npv, sensitivity, specificity, mcc,f1, roc_auc))
    cv_clf.save(f'{fldr_path}/' + str(i) + 'DNN_model.keras')


scores=np.array(sepscores)
print("acc=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[0]*100,np.std(scores, axis=0)[0]*100))
print("precision=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[1]*100,np.std(scores, axis=0)[1]*100))
print("npv=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[2]*100,np.std(scores, axis=0)[2]*100))
print("sensitivity=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[3]*100,np.std(scores, axis=0)[3]*100))
print("specificity=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[4]*100,np.std(scores, axis=0)[4]*100))
print("mcc=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[5]*100,np.std(scores, axis=0)[5]*100))
print("f1=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[6]*100,np.std(scores, axis=0)[6]*100))
print("roc_auc=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[7]*100,np.std(scores, axis=0)[7]*100))
result1=np.mean(scores,axis=0)
H1=result1.tolist()
sepscores.append(H1)
result=sepscores
row=yscore.shape[0]
yscore=yscore[np.array(range(1,row)),:]
yscore_sum = pd.DataFrame(data=yscore)
yscore_sum.to_csv(f'{fldr_path}/DNN_yscore.csv')
ytest=ytest[np.array(range(1,row)),:]
ytest_sum = pd.DataFrame(data=ytest)
ytest_sum.to_csv(f'{fldr_path}/DNN_ytest.csv')

data_csv = pd.DataFrame(data=result)
data_csv.to_csv(f'{fldr_path}/DNN_CV_results.csv')


(996, 326)
Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.6478 - loss: 0.6578
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8770 - loss: 0.2617
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9517 - loss: 0.1293
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9632 - loss: 0.1153
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9765 - loss: 0.0589
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9844 - loss: 0.0542
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9854 - loss: 0.0284
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9939 - loss: 0.0251
Epoch 9/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9945 - loss: 0.0205
Epoch 10/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9885 - loss: 0.0248
Epoch 11/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9909 - loss: 0.0231
Epoch 12/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
DNN:acc=0.944724,precision=0.978261,npv=0.915888,sensitivity=0.909091,specificity=0.980000,mcc=0.891616,f1=0.942408,roc_auc=0.985101
Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.6088 - loss: 0.7104
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9082 - loss: 0.2443
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9283 - loss: 0.1651
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9530 - loss: 0.1290
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9749 - loss: 0.0790
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9871 - loss: 0.0457
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9780 - loss: 0.0611
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9862 - loss: 0.0349
Epoch 9/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9812 - loss: 0.0405
Epoch 10/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
DNN:acc=0.934673,precision=0.899083,npv=0.977778,sensitivity=0.980000,specificity=0.888889,mcc=0.872866,f1=0.937799,roc_auc=0.985556
Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6328 - loss: 0.6794
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8950 - loss: 0.2554
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9317 - loss: 0.1638
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9488 - loss: 0.1189
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9618 - loss: 0.1247
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9814 - loss: 0.0544
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9875 - loss: 0.0363
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9819 - loss: 0.0493
Epoch 9/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9848 - loss: 0.0241
Epoch 10/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9

**Indpendent test**

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.models import Sequential, load_model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
import math

path = "/content/drive/MyDrive/Watashara_Projects/ACE/"
# data_ = pd.read_csv(path + 'Dts_AEDN1000.csv')
# data = data_.iloc[:, :-1].values
# labels = data_.iloc[:, -1].values
# y = labels
# shu = scale(data)
# Xt = shu
# yt = labels
# X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

folder_name = "Update_results/DNN_ACE"

test_feat = pd.read_csv(path + 'selected_features_allfeat/XtestData.csv')


Xt = test_feat.iloc[:, :-1].values
yt = test_feat.iloc[:, -1].values



print(Xt.shape)
print(yt.shape)

# Xt = X_ind
# yt = y_ind
#

###########################################################################################

[m,n]=np.shape(Xt)


sepscores = []
ytest = np.ones((1, 2)) * 0.5
yscore = np.ones((1, 2)) * 0.5

for i in range(5):

    loaded_model = load_model(path+ f'{folder_name}/' + str(i) + 'DNN_model.keras')
    print("Loaded model from disk")

    # evaluate loaded model on test data

    y_score = loaded_model.predict(Xt)
    y_class = categorical_probas_to_classes(y_score)

    y_test = to_categorical(yt)
    ytest = np.vstack((ytest, y_test))
    y_test_tmp = yt
    yscore = np.vstack((yscore, y_score))

    acc, precision, npv, sensitivity, specificity, mcc, f1 = calculate_performace(len(y_class), y_class, y_test_tmp)
    fpr, tpr, _ = roc_curve(y_test[:, 1], y_score[:, 1])
    roc_auc = auc(fpr, tpr)
    sepscores.append([acc, sensitivity, specificity, mcc, f1, roc_auc])
    print(f'CNN:acc={acc:.6f},sensitivity={sensitivity:.6f},specificity={specificity:.6f},mcc={mcc:.6f},f1={f1:.6f},roc_auc={roc_auc:.6f}')


row=ytest.shape[0]
ytest=ytest[np.array(range(1,row)),:]
ytest_sum = pd.DataFrame(data=ytest)
ytest_sum.to_csv(path+ f'{folder_name}/DNN_ytest_test.csv')

yscore_=yscore[np.array(range(1,row)),:]
yscore_sum = pd.DataFrame(data=yscore_)
yscore_sum.to_csv(path+ f'{folder_name}/DNN_yscore_test.csv')

scores = np.array(sepscores)
result1 = np.mean(scores, axis=0)
H1 = result1.tolist()
sepscores.append(H1)
result = sepscores

data_csv = pd.DataFrame(data=result)
data_csv.to_csv(path+ f'{folder_name}/DNN_test_results.csv')





(204, 325)
(204,)
Loaded model from disk
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step
CNN:acc=0.882353,sensitivity=0.842105,specificity=0.906250,mcc=0.748355,f1=0.842105,roc_auc=0.954359
Loaded model from disk
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
CNN:acc=0.911765,sensitivity=0.855263,specificity=0.945312,mcc=0.809955,f1=0.878378,roc_auc=0.952611
Loaded model from disk
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
CNN:acc=0.911765,sensitivity=0.776316,specificity=0.992187,mcc=0.815456,f1=0.867647,roc_auc=0.965563
Loaded model from disk
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
CNN:acc=0.897059,sensitivity=0.881579,specificity=0.906250,mcc=0.781943,f1=0.864516,roc_auc=0.966797
Loaded model from disk
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
CNN:acc=0.921569,sensitivity=0.855263,specificity=0.960937,mcc=0.831203,f1=0.890411,roc_auc=0.970189
